# PyTorch

## MeuralNetworks

### Imports

In [38]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

### Get Device

look if there is a Hardware piece that is faster than cpu, if there is no ``cuda`` device, continue with ``cpu``

In [39]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


### Define Class

a class that implements ``nn.Module``

in the constructor we define the Layers

In [40]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

generating a Neural Network on the ``device``

In [41]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Throwing Data into the Model

In [42]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([1], device='cuda:0')


### Layers

#### Visualisation Data

```torch.rand(number of Tensors,height,width)```

In [43]:
input_image = torch.rand(3,28,28)
print(input_image)

tensor([[[0.7736, 0.6059, 0.2049,  ..., 0.1303, 0.6702, 0.6640],
         [0.2022, 0.6409, 0.3240,  ..., 0.4313, 0.9897, 0.4752],
         [0.1571, 0.0589, 0.4267,  ..., 0.7221, 0.1706, 0.0165],
         ...,
         [0.0677, 0.8477, 0.8631,  ..., 0.5944, 0.7018, 0.9786],
         [0.3240, 0.7030, 0.8383,  ..., 0.8467, 0.4085, 0.3515],
         [0.3371, 0.7567, 0.6750,  ..., 0.6946, 0.6174, 0.6412]],

        [[0.0459, 0.0632, 0.9060,  ..., 0.2900, 0.0061, 0.1192],
         [0.4834, 0.4391, 0.8833,  ..., 0.0897, 0.1878, 0.8509],
         [0.7655, 0.4648, 0.8997,  ..., 0.3324, 0.4053, 0.4675],
         ...,
         [0.1109, 0.8367, 0.6119,  ..., 0.5624, 0.7419, 0.7576],
         [0.5693, 0.0913, 0.4346,  ..., 0.5071, 0.1803, 0.5837],
         [0.9799, 0.0607, 0.0637,  ..., 0.9835, 0.8182, 0.7734]],

        [[0.8419, 0.5577, 0.4350,  ..., 0.2208, 0.4074, 0.7828],
         [0.6662, 0.4550, 0.0205,  ..., 0.6755, 0.2902, 0.5380],
         [0.2710, 0.5375, 0.9313,  ..., 0.9471, 0.2428, 0.

In [44]:
input_image.size()

torch.Size([3, 28, 28])

#### nn.Flatten

turns 2D data to 1D data

``[1, 1], [2, 2]  -> [1, 1, 2, 2]``

In [45]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


#### nn.Linear

linear transformation on the input using its stored weights and biases

In [46]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


#### nn.ReLU

non-linear activations. These do the complex maping inside the NN. There are other methodes than ReLU to do these actions

In [50]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1.size()}")

Before ReLU: tensor([[0.3956, 0.4323, 0.0000, 0.0000, 0.1807, 0.0000, 0.0000, 0.0000, 0.0267,
         0.0581, 0.0000, 0.1046, 0.0000, 0.0000, 0.1178, 0.1602, 0.0487, 0.3046,
         0.0000, 0.3459],
        [0.0000, 0.0562, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.3084,
         0.0660, 0.0000, 0.3950, 0.0000, 0.0000, 0.4076, 0.0000, 0.0000, 0.4734,
         0.0000, 0.3816],
        [0.3046, 0.4821, 0.0926, 0.0000, 0.0000, 0.0000, 0.1869, 0.0000, 0.3388,
         0.0000, 0.0139, 0.0922, 0.0000, 0.0729, 0.1813, 0.0000, 0.0948, 0.1120,
         0.0642, 0.3662]], grad_fn=<ReluBackward0>)


After ReLU: torch.Size([3, 20])


#### nn.Sequential

a collection of Layertypes

<img title="nn.Sequential" alt="Picture" src="res/seq.PNG" height="300">

In [48]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

#### nn.Softmax

Last Layor. Returns an evaluation of Likelyhood of every Output in an Intervall [0,1].

In [49]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

#### Visualization

<img title="Layers drawn" alt="Picture" src="res/layers.PNG">

### Model Parameters

every layer has a parameter or a bias

````parameters()```` or ````named_parameters()````